# 使用 ppyolov2 来做钢铁缺陷检测
只需更改一些配置文件，跑通模型，就可以超过基线！

In [ ]:
!unzip /home/aistudio/steel_bug_detect.zip

In [ ]:
# paddle环境准备
!git clone https://gitee.com/paddlepaddle/PaddleDetection.git
%cd PaddleDetection
# 安装其他依赖
! pip install paddledet==2.0.1 -i https://mirror.baidu.com/pypi/simple
! pip install -r requirements.txt

## 注意
/home/aistudio/ppyolov2_r50vd_dcn_voc 这个文件里存着我之前训练的checkpoints，将文件转移到 /home/aistudio/PaddleDetection/output/ 下，这样可以接着训练或者直接预测

In [ ]:
!mv /home/aistudio/ppyolov2_r50vd_dcn_voc /home/aistudio/PaddleDetection/output/ 

# 主要记得修改配置文件：

> - steel_bug_detect/train/ppyolov2_r50vd_dcn_voc.yml 这是主配置文件，下面的文件路径都放在这个开头了
> - steel_bug_detect/train/\_base_/optimizer_365e.yml ：修改epoch数
> - steel_bug_detect/train/runtime.yml ：如果嫌打印的太多了，可以在这里修改 snapshot_epoch 数
> - steel_bug_detect/train/\_base_/ppyolov2_r50vd_dcn.yml ：修改 anchors：
[[18,36],[45,36],[24,71],
            [16,169],[53,69],[38,189],
            [138,62],[76,123],[154,192]]
> - steel_bug_detect/train/\_base_/ppyolov2_reader.yml : 修改同样的 anchors （anchors由steel_bug_detect/train/kmeans-anchor-boxes.py 计算得出）

# 训练
从头开始训练的话就只要使用
```
!python ./tools/train.py -c ../steel_bug_detect/train/ppyolov2_r50vd_dcn_voc.yml --eval --use_vdl=True
```
就可以了，如果想接着之前的训练，最后要加上
```
 -r <PATH_OF_YOUR_LAST_CHECKPOINT>
```
checkpoints会存放在 /home/aistudio/PaddleDetection/output/ppyolov2_r50vd_dcn_voc/ 路径里

In [4]:
# 使用train.py进行训练
# !python ./tools/train.py -c ../steel_bug_detect/train/yolov3_mobilenet_v1_270e_voc.yml --eval --use_vdl=True
#!python ./tools/train.py -c ../steel_bug_detect/train/yolov3_mobilenet_v1_270e_voc.yml -r ./output/yolov3_mobilenet_v1_270e_voc/model_final.pdparams --eval --use_vdl=True

# !python ./tools/train.py -c ../steel_bug_detect/train/ppyolov2_r50vd_dcn_voc.yml --eval -r ./output/ppyolov2_r50vd_dcn_voc/169.pdparams --use_vdl=True
!python ./tools/train.py -c ../steel_bug_detect/train/ppyolov2_r50vd_dcn_voc.yml --eval --use_vdl=True -r ./output/ppyolov2_r50vd_dcn_voc/model_final.pdparams

## ‘optional’ 可以使用visualdl来可视化一下

In [4]:
!pip install visualdl==2.0.4
!visualdl service upload --model PaddleDetection/vdl_log_dir/scalar/vdlrecords.1637202313.log

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Uploading file `PaddleDetection/vdl_log_dir/scalar/vdlrecords.1637202313.log`
View your visualization results at: `https://paddlepaddle.org.cn/paddle/visualdl/service/app?id=1e7509a6a5189bb1eb1b5c47ff7916f8`.


# 使用训练好的模型来测试

记得加上 --save_txt=True 这是咱的预测后的结果文件，

记得把 --draw_threshold 后的数值调低一些，这样检测出的结果更多，打比赛得分会变高

In [12]:
# 预测
# !python ./tools/infer.py -c ../steel_bug_detect/train/yolov3_mobilenet_v1_270e_voc.yml -o weights=./output/yolov3_mobilenet_v1_270e_voc/best_model.pdparams --infer_dir=../steel_bug_detect/test/IMAGES --output_dir=../steel_bug_detect/test/yolov3_mobilenet_result --save_txt=True --draw_threshold 0.3
#!python ./tools/infer.py -c ../steel_bug_detect/train/ppyolov2_r50vd_dcn_voc.yml -o weights=./output/ppyolov2_r50vd_dcn_voc/best_model.pdparams --infer_dir=../steel_bug_detect/test/IMAGES
!python ./tools/infer.py -c ../steel_bug_detect/train/ppyolov2_r50vd_dcn_voc.yml -o weights=./output/ppyolov2_r50vd_dcn_voc/369.pdparams --infer_dir=../steel_bug_detect/test/IMAGES --output_dir=../steel_bug_detect/test/ppyolov2_result --save_txt=True --draw_threshold 0.005

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the

In [6]:
#!pwd
%cd PaddleDetection

/home/aistudio/PaddleDetection


# 最后的结果文件csv一定要是按 image id顺序排列，否则比赛得分会很低

In [11]:
import os
import pandas as pd

result = pd.DataFrame(columns=['image_id', 'bbox', 'category_id', 'confidence'])
result_map = {'crazing':0,'inclusion':1, 'pitted_surface':2, 'scratches':3, 'patches':4, 'rolled-in_scale':5}
file_path = '/home/aistudio/steel_bug_detect/test/ppyolov2_result/'

def work(temp):
    global result
    with open(file_path + temp, 'r') as fi:
        lines = fi.readlines()
    #result.split(' ')

    image_id = temp.replace('.jpg.txt', '')
    image_id = temp.replace('.txt', '')

    #image_id
    #print('lines=', lines)
    for line in lines:
        #xmin, ymin, xmax, ymax, category_id, confidence, _ = line.split(' ')
        line = line.replace('\n', '')
        category, confidence, xmin, ymin, w, h = line.split(' ')
        category_id = result_map[category]
        
        xmin, ymin, xmax, ymax = int(float(xmin)), int(float(ymin)), int(float(xmin)+float(w)), int(float(ymin)+float(h))
        if xmin > xmax:
            z = xmin
            xmin = xmax
            xmax = z
        if ymin > ymax:
            z = ymin
            ymin = ymax
            ymax = z
        if ymin <= 0:
            ymin = 1
        bbox = [xmin, ymin, xmax, ymax]
        temp_result = {}
        temp_result['image_id'] = image_id
        temp_result['bbox'] = bbox
        temp_result['category_id'] = category_id
        temp_result['confidence'] = confidence
        result = result.append(temp_result, ignore_index=True)

    #print(xmin, ymin, xmax, ymax, category_id, confidence)
for temp in os.listdir(file_path):
    if temp[-4:] == '.txt':
        work(temp)
result = result.sort_values(by=['image_id'])
result.to_csv('ppyolov2_result_369e_0003.csv', index=False)
result

In [ ]:
# 也可以试试别的模型来预测
# !python ./tools/infer.py -c ../steel_bug_detect/train/yolov3_mobilenet_v1_270e_voc.yml -o weights=./output/yolov3_mobilenet_v1_270e_voc/best_model.pdparams --infer_dir=../steel_bug_detect/test/IMAGES --save_txt True

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the